# Hotel Recommendation Using NLP

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle datasets download -d jiashenliu/515k-hotel-reviews-data-in-europe

 84% 38.0M/45.1M [00:00<00:00, 106MB/s] 
100% 45.1M/45.1M [00:00<00:00, 98.5MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

Archive:  515k-hotel-reviews-data-in-europe.zip
  inflating: Hotel_Reviews.csv       


In [ ]:
import string
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = pd.read_csv('Hotel_Reviews.csv')
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [ ]:
df.columns=[x.lower() for x in df.columns]

In [ ]:
df.columns

Index(['hotel_address', 'additional_number_of_scoring', 'review_date',
       'average_score', 'hotel_name', 'reviewer_nationality',
       'negative_review', 'review_total_negative_word_counts',
       'total_number_of_reviews', 'positive_review',
       'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'reviewer_score', 'tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [ ]:
drop_columns = ['additional_number_of_scoring',
                'review_date', 'reviewer_nationality',
                'negative_review', 'review_total_negative_word_counts',
                'total_number_of_reviews', 'positive_review',
                'review_total_positive_word_counts',
                'total_number_of_reviews_reviewer_has_given',
                'reviewer_score', 'days_since_review']
df1 = df.drop(columns = drop_columns, axis = 1)
df1.head()

,hotel_address,average_score,hotel_name,tags,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",52.360576,4.915968


In [ ]:
df1.isna().sum()

hotel_address       0
average_score       0
hotel_name          0
tags                0
lat              3268
lng              3268
dtype: int64

In [ ]:
df1.dropna(inplace = True)

In [ ]:
print('Duplicated data:', df1.duplicated().sum())

Duplicated data: 334972


In [ ]:
df1.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
hotel=list(df1['hotel_name'].unique())
len(hotel)

1475

In [ ]:
# Inherently, there are "H tel" in the dataset
df1[df1['hotel_name'].str.contains("H tel")]['hotel_name'].unique()

array(['H tel De Vend me', 'H tel des Ducs D Anjou',
       'H tel Juliana Paris', 'H tel de Jos phine BONAPARTE',
       'H tel Keppler', 'H tel Chaplain Paris Rive Gauche',
       'H tel Regina Op ra Grands Boulevards', 'H tel Diva Opera',
       'H tel Duo', 'H tel Le Marianne', 'H tel de la Bourdonnais',
       'Terrass H tel Montmartre by MH', 'H tel Square Louvois',
       'H tel San R gis', 'H tel Westminster', 'L H tel',
       'H tel Duc De St Simon', 'H tel Madison by MH',
       'H tel des Academies et des Arts', 'Mill sime H tel',
       'H tel California Champs Elys es', 'H tel Hor',
       'H tel de Banville', 'H tel Raphael', 'H tel Bedford',
       'H tel Horset Op ra Best Western Premier Collection',
       'H tel Les Dames du Panth on', 'H tel Victor Hugo Paris Kl ber',
       'H tel Ad le Jules', 'H tel Regina', 'H tel des Champs Elys es',
       'Lyric H tel Paris', 'H tel Belloy Saint Germain By Happyculture',
       'H tel Le M', 'Banke H tel', 'H tel F licien by 

In [ ]:
#impute the word "H tel" to "Hotel"
df1['hotel_name']=df['hotel_name'].apply(lambda x:x.replace('H tel','Hotel'))

In [ ]:
# Replacing "United Kingdom with "UK"
df1['hotel_address'] = df1['hotel_address'].str.replace("United Kingdom", "UK")

In [ ]:
# Split the address and pick the last word in the address to identify the country
df1["countries"] = df1['hotel_address'].apply(lambda x: x.split(' ')[-1])
print(df1['countries'].unique())

['Netherlands' 'UK' 'France' 'Spain' 'Italy' 'Austria']


In [ ]:
exclude = set(string.punctuation)
def clean(x):
    return set([''.join(ch for ch in i.lower() if ch not in exclude).strip() for i in x[2:][:-2].split(',')])

In [ ]:
# applying to clean the tags and assigning them to a new column
df1['tags'] = df1['tags'].map(clean)

In [ ]:
# global variable
tag_sum_list = []

def get_tag_sum_elems(tag_sum_string):
    global tag_sum_list # use the global variable
    # extend the global variable with this_list
    tag_sum_list.extend(tag_sum_string)
    return True

for i in df1['tags']:
    get_tag_sum_elems(i)

tag_sum_set = set(tag_sum_list)
tag_sum_set

{'basic twin room',
 'deluxe double with winter garden view',
 'deluxe double and single room',
 'superior double or twin room with terrace',
 'superior double room with queen size bed',
 'luxury suite superior',
 'embassy suite',
 'deluxe double or twin room with spa bath',
 'superior double room 3 adults',
 'superior suite 3 adults',
 'superior room pigalle',
 'king suite non smoking with executive lounge access',
 'executive suite 3 adults',
 'stayed 27 nights',
 'deluxe double room with two double beds',
 'superior junior suite',
 'accessible twin room',
 'presidential suite with view terrace',
 'apartment 4 adults',
 'k suite',
 'superior twin room barlow wing',
 'triple room with terrace',
 'executive twin room',
 'suite tea time wine tasting included',
 'double room disability access tub',
 'deluxe duplex room',
 'superior suite water front',
 'standard double twin room',
 'business trip',
 'park deluxe view twin',
 'twin grand premier with canal view',
 'deluxe double room with

In [ ]:
df1.head()

,hotel_address,average_score,hotel_name,tags,lat,lng,countries
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"{leisure trip, couple, duplex double room, sta...",52.360576,4.915968,Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"{stayed 4 nights, leisure trip, couple, duplex...",52.360576,4.915968,Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"{family with young children, submitted from a ...",52.360576,4.915968,Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"{stayed 3 nights, leisure trip, duplex double ...",52.360576,4.915968,Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,7.7,Hotel Arena,"{stayed 2 nights, couple, submitted from a mob...",52.360576,4.915968,Netherlands


In [ ]:
print(df1.shape)

(177498, 7)


In [ ]:
# Function to remove the 'submitted from a mobile device' tag from the tags set
def remove_tag(tags_set):
    tags_set.discard("submitted from a mobile device")
    return tags_set

In [ ]:
# Apply the function to the 'tags' column
df1['tags'] = df1['tags'].apply(remove_tag)

In [ ]:
def recommend_hotel(input_description, data=df1):
    # Preprocess the input
    input_description = input_description.lower()
    words = word_tokenize(input_description)
    stop_words = stopwords.words('english')
    lemm = WordNetLemmatizer()
    filtered = [lemm.lemmatize(word) for word in words if word not in stop_words and word.isalpha()]

    # Extract words from the input description as potential location identifiers
    potential = set(filtered)

    # Prepare the data
    data['processed_address'] = data['hotel_address'].apply(lambda x: set(word_tokenize(x.lower())))
    data['processed_tags'] = data['tags'].apply(lambda x: set(word_tokenize(x.lower())) if isinstance(x, str) else set())

    # Compute similarity based on tags and check for location match in address
    cos = []
    for index, row in data.iterrows():
        tag_intersection = row['processed_tags'].intersection(potential)
        address_intersection = row['processed_address'].intersection(potential)
        # Increase similarity score for matches in address to prioritize location match
        cos.append(len(tag_intersection) + len(address_intersection))

    data['similarity'] = cos
    data = data.sort_values(by=['similarity', 'average_score'], ascending=[False, False])
    data = data.drop_duplicates(subset='hotel_name', keep='first')
    data.reset_index(drop=True, inplace=True)

    return data[['hotel_name', 'hotel_address', 'average_score']].head()

In [ ]:
input_description = input("What do you need?\n")
recommendation = recommend_hotel(input_description)
recommendation

What do you need?
I am going on a business trip on a budget, so I need a classic room for 5 nights in Chelsea, London.


,hotel_name,hotel_address,average_score
0,Milestone Hotel Kensington,1 Kensington Court Kensington and Chelsea Lond...,9.5
1,Egerton House,17 19 Egerton Terrace Knightsbridge Kensington...,9.3
2,The Hari London,20 Chesham Place Kensington and Chelsea London...,9.2
3,Draycott Hotel,26 Cadogan Gardens Kensington and Chelsea Lond...,9.2
4,Knightsbridge Hotel,10 Beaufort Gardens Kensington and Chelsea Lon...,9.1


In [ ]:
input_description = input("What do you need?\n")
recommendation = recommend_hotel(input_description)
recommendation

What do you need?
I am going for honeymoon with my beloved and need a luxury room for 3 nights in Paris.


,hotel_name,hotel_address,average_score
0,Ritz Paris,15 Place Vend me 1st arr 75001 Paris France,9.8
1,Hotel de La Tamise Esprit de France,4 rue d Alger 1st arr 75001 Paris France,9.6
2,Hotel The Peninsula Paris,19 avenue Kleber 16th arr 75116 Paris France,9.5
3,Le Narcisse Blanc Spa,19 Boulevard De La Tour Maubourg 7th arr 75007...,9.5
4,Nolinski Paris,16 Avenue de l Opera 1st arr 75001 Paris France,9.4
